In [1]:
import os
import re
import pymysql
import calendar
import numpy as np
import pandas as pd
from pypinyin import Style, pinyin

# 连接<理赔打假>数据库
conn = pymysql.connect(host='localhost',user = "root",passwd = "gu711205",db='lp_dj',charset = 'gbk')
cursor = conn.cursor()

In [2]:
# 工具函数
def bopomofo_joint(a):
    '''
    连接中文拼音
    '''
    x = ''
    for i in a:
        x += i[0]
    return x.upper()

def save(filename, docs):
    '''
    存储填数SQL语句
    '''
    fh = open(filename, 'w', encoding='utf-8')
    for doc in docs:
        fh.write(doc)
        fh.write('\n')
    fh.close()
    
def isNull(values):
    '''
    填数SQL语句空值处理方法
    '''
    error_type = ['nan', 'NaN', 'NaT']
    if str(values) in error_type:
        output = 'NULL,'
    else:
        output = '\'' + str(values) + '\','
    return output

In [3]:
# 数据库中的表名
table_name = 'lp_ml_saccvt_deduction_cla_tp_fgs'
# 读取业务数据
file_name = '车理赔老旧车型赔案_安徽分公司_201901-202012.xlsx'
df = pd.read_excel(file_name, skiprows=3)

# 更改默认路径
os.chdir(os.path.abspath(os.path.dirname(os.getcwd())) + '/已建数据库结构的SQL文件')
# 读取<规范表>
df_address = pd.read_excel('业务数据字段命名及设计规范.xlsx')
df_address.columns = ['中文名', '英文名', 'SQL语句']
dict_address = dict(zip(df_address['中文名'],df_address['SQL语句']))

#### 对于新增字段，我们默认设置为VARCHAR(255)格式：理应在事后查明这些新增字段的建表格式，填入<业务数据字段命名及设计规范.xlsx>  也即(<维护表>)中。

In [4]:
# 根据表头建库
sqltxt = [ 'CREATE TABLE ' + table_name + '(\n', "    ID INT NOT NULL AUTO_INCREMENT  COMMENT 'ID' ,\n", "    DATA_DATE DATE    COMMENT '数据时间' ,\n", "    REVISION INT    COMMENT '乐观锁' ,\n", "    CREATED_BY VARCHAR(32)    COMMENT '创建人' ,\n", "    CREATED_TIME DATETIME    COMMENT '创建时间' ,\n", "    UPDATED_BY VARCHAR(32)    COMMENT '更新人' ,\n", "    UPDATED_TIME DATETIME    COMMENT '更新时间' ,\n"]
for col_name in df.columns:
    
    if col_name in dict_address.keys():
        sqltxt.append(dict_address[col_name])
    else:
        print("新增字段名: ", col_name)
        type_name = 'VARCHAR(255)'
        sentence = '    ' + bopomofo_joint(pinyin(col_name, style=Style.NORMAL)) + "   " + type_name + "     COMMENT '" + col_name + "' ,\n"
        sqltxt.append(sentence)
        dict_address[col_name] = sentence
        
sqltxt.append("    PRIMARY KEY (ID)\n) COMMENT = '" + file_name.split('.')[0] + " ';;")

新增字段名:  子险代码
新增字段名:  子险名称
新增字段名:  支付对象代码
新增字段名:  车损险赔款
新增字段名:  车损险保额


In [5]:
# 更改存储路径
os.chdir(os.path.abspath(os.path.dirname(os.getcwd())) + '/根据Excel表自动创建数据库/已创建数据表备份存储')
# 存储创建表的SQL语句
with open(file_name.split('.')[0]+".sql","w") as f:
    for s in sqltxt:
        f.write(s)

In [6]:
#创建表
creatTable_sql = "drop table if exists " + table_name
cursor.execute(creatTable_sql)
sql_path = file_name.split('.')[0] + ".sql"

# 读取 sql 文件文本内容
sql = open(sql_path, "r", encoding = "utf8")
sqltxt = sql.readlines()# 此时 sqltxt 为 list 类型

# 读取之后关闭文件
sql.close()

# list 转 str
sql = "".join(sqltxt)

# 执行sql
cursor.execute(sql)
print('创建数据库成功!')

创建数据库成功!


#### 测试插入<车理赔老旧车型赔案_安徽分公司_201901-202012>的数据于自动生成的Table中:

In [7]:
Table_Header = pd.DataFrame({'ID':np.arange(len(df)) + 1, 'Data_Date':['2020-01-01'] * len(df)})
df = pd.concat([Table_Header,df], axis=1)

In [8]:
exec_code_ls = []
cols_len = len(df.iloc[0])

for j in range(len(df)):
    col_values = 'INSERT INTO ' + table_name + ' values('
    # 将"乐观锁",“创建人”,“创建时间”,“更新人”,“更新时间列”设置为空值
    for i in range(cols_len):
        if i == 2:
            col_values += 'NULL,'*5
        
        col_values += isNull(df.iloc[j][i])
          
    # 去除最后一个逗号
    col_values = col_values[:-1]
    # 加上后缀
    col_values += ');'
    exec_code_ls.append(col_values)
    #del col_values

In [9]:
for i in range(len(exec_code_ls)):
    sql = exec_code_ls[i]
    try:
       # 执行sql语句
       cursor.execute(sql)
       # 提交到数据库执行
       conn.commit()
    
    except:
       # 如果发生错误则回滚
       conn.rollback()
        
#conn.close()  

#### 测试代码，显示入库提数成功。要执行需注释上"conn.close()"语句

In [10]:
test_sql = 'select * from lp_ml_saccvt_deduction_cla_tp_fgs'
cursor.execute(test_sql)
data = cursor.fetchall()
print(data)

((1, datetime.date(2020, 1, 1), None, None, None, None, None, '201901', '安徽分公司', '合肥中心支公司', '综合业务部', '11024400', '机动车商业行业示范汽车保险', 'AHEFHF1Y1418B017886E', None, None, datetime.date(2018, 4, 2), datetime.datetime(2019, 4, 1, 0, 0), Decimal('1076768.00000000'), Decimal('4011.75000000'), '皖A87V50', 'LVSHBFMC4BF188531', '蒙迪欧CAF7203M轿车', datetime.date(2011, 9, 21), 7, '340123199112252077', '吴海洲', '340123199112252077', '吴海洲', None, None, 'C340100VEH18038623', '皖A87V50 标的车：皖A87V50右侧受损，撞非机动车/固定物/动物，碰三者物损：护栏有损，已报警或提示报警；现', None, None, None, '否', '否', None, None, None, '安徽省滁州市定远县合徐高速', None, '吴海洲', 'F340100A1418036692', 'F340100A1418036692_4', Decimal('64435.00000000'), Decimal('0E-8'), datetime.datetime(2019, 1, 9, 0, 0), '0', '吴海洲', Decimal('64435.00000000'), '徽商银行股份有限公司阜阳颍东支行', '6217751020000257413', datetime.date(2019, 1, 10), '54560', '76768', 'AH3401000191', '合肥恒信德龙美华汽车销售服务有限公司', 'AH3401000191', '合肥恒信德龙美华汽车销售服务有限公司', None, None, None, None, 'HF796', '管如晟', None, None, 'BB224', '陈子明', 'HF196